# Implement Bert

Note: It takes 485  seconds to run 100 abstracts. We are working with 690,815 abstract. 

In [1]:
#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd

#import re

import nltk
#nltk.download("punkt") #input: punkt

from nltk import tokenize

import statistics #calculate mean and others

import datetime

In [2]:
# comparison see https://www.sbert.net/docs/pretrained_models.html
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2') #good quality, shorter run time
#embedder = SentenceTransformer('paraphrase-mpnet-base-v2') #takes longer to run, but the best quality

In [3]:
abstracts = pd.read_pickle("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/smaller-final-dataset.pkl")

In [4]:
abstracts

,original index,PROJECT_ID,ABSTRACT,FY,ORG_COUNT,PI_COUNT,nchar,final_frqwds_removed,PROJECT_TERMS,PROJECT_TITLE,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,1,1,1402,"[multiprotein, y_secretase, proteolytically_cl...",Active Sites; Affect; Alzheimer's Disease; Am...,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,...,BOSTON,MA,021156110,UNITED STATES,12/1/2007,1/1/2008,93.866,2008,3483.0,NaN
1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,1,1,2553,"[kissl, gene, encode, peptide, kisspeptin, bin...",Affect; Animal Model; Axon; Behavior; Behavio...,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,...,SEATTLE,WA,981959472,UNITED STATES,9/1/2008,1/1/2009,93.865,2008,39175.0,NaN
2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,1,1,1414,"[biophysical, basis, thermodynamics_kinetic, m...",Agreement; Antibodies; base; Binding; Biochem...,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,...,CAMBRIDGE,MA,021385319,UNITED STATES,1/2/2008,1/1/2009,93.859,2008,49646.0,NaN
3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,1,1,1545,"[obesity, adverse_pregnancyoutcome, great, hea...",African; Analysis of Variance; Asians; Birth;...,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,...,HOUSTON,TX,770305400,UNITED STATES,4/1/2008,1/1/2009,93.361,2008,20406.0,NaN
4,371628,594482,Local potato advisory groups have expressed in...,2010,1,1,271,"[local, potato, advisory, express, interest, m...",cost; Health; interest; Manure; Parasitic nem...,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,...,CORVALLIS,OR,97331,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690809,1034758,1126666,Project Summary/AbstractThe pathophysiology of...,2019,1,1,2836,"[pathophysiology, schizophrenia, advance, thed...",Adolescence; Adult; Adult Children; Affect; A...,GUT DYSBIOSIS AND 5-HT2A DYSREGULATION IN A PR...,...,RICHMOND,VA,232980568,UNITED STATES,9/10/2019,9/9/2020,93.242,2019,37602.0,NaN
690810,1074034,1109788,PROJECT SUMMARY:Alzheimer’s Disease (AD) and A...,2019,1,1,2899,"[alzheimer, ad, amyotrophic_lateral_sclerosis_...",abeta deposition; Acetylgalactosamine; Affect...,TISSUE-SPECIFIC MODULATION OF APOLIPOPROTEIN E...,...,WORCESTER,MA,016550002,UNITED STATES,9/10/2019,9/9/2020,93.866,2019,32209.0,NaN
690811,1029450,1038442,"Project SummaryThe highest rates of mortality,...",2018,1,1,2186,"[highest, mortality, acute, care, encounter, r...",Acute; Address; Adherence; Adolescent; Adoles...,THE FEASIBILITY OF A MEDICAL STUDENT MENTORING...,...,NEW BRUNSWICK,NJ,089011914,UNITED STATES,9/10/2018,9/9/2019,93.840,2018,34994.0,NaN
690812,1074409,1154490,PROJECT SUMMARY: Treatment paradigms for kidne...,2019,1,1,3056,"[paradigm, kidney, largely, stagnant, decade, ...",3-Dimensional; Acute; Acute Renal Failure wit...,STRATEGIES IN RENAL NANOMEDICINE TO IMPACT TRE...,...,CHICAGO,IL,606113152,UNITED STATES,9/10/2019,9/9/2020,93.847,2019,50016.0,NaN


In [12]:
abstracts_try = abstracts.iloc[0:100]
abstracts_try

,original index,PROJECT_ID,ABSTRACT,FY,ORG_COUNT,PI_COUNT,nchar,final_frqwds_removed,PROJECT_TERMS,PROJECT_TITLE,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,1,1,1402,"[multiprotein, y_secretase, proteolytically_cl...",Active Sites; Affect; Alzheimer's Disease; Am...,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,...,BOSTON,MA,021156110,UNITED STATES,12/1/2007,1/1/2008,93.866,2008,3483.0,NaN
1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,1,1,2553,"[kissl, gene, encode, peptide, kisspeptin, bin...",Affect; Animal Model; Axon; Behavior; Behavio...,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,...,SEATTLE,WA,981959472,UNITED STATES,9/1/2008,1/1/2009,93.865,2008,39175.0,NaN
2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,1,1,1414,"[biophysical, basis, thermodynamics_kinetic, m...",Agreement; Antibodies; base; Binding; Biochem...,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,...,CAMBRIDGE,MA,021385319,UNITED STATES,1/2/2008,1/1/2009,93.859,2008,49646.0,NaN
3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,1,1,1545,"[obesity, adverse_pregnancyoutcome, great, hea...",African; Analysis of Variance; Asians; Birth;...,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,...,HOUSTON,TX,770305400,UNITED STATES,4/1/2008,1/1/2009,93.361,2008,20406.0,NaN
4,371628,594482,Local potato advisory groups have expressed in...,2010,1,1,271,"[local, potato, advisory, express, interest, m...",cost; Health; interest; Manure; Parasitic nem...,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,...,CORVALLIS,OR,97331,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,758240,741964,DESCRIPTION (provided by applicant): Neuro-inf...,2015,1,1,1724,"[neuro_inflammation, immunosuppression, gliobl...",activating transcription factor; Affect; angi...,ROLE AND THERAPEUTIC VALUE OF AHR IN INFLAMMAT...,...,BOSTON,MA,021156110,UNITED STATES,5/1/2015,1/1/2016,93.853,2015,1.0,NaN
96,839161,915408,To determine the incidence and severity of sle...,2016,1,1,270,"[incidence, severity, sleep_apnea, distinguish...",cognitive function; Incidence; indexing; Medi...,MEDIATION EFFECTS OF SLEEP APNEA ON INDICES OF...,...,SILVER SPRING,MD,20910,UNITED STATES,NaN,NaN,NaN,2016,NaN,NaN
97,884503,861883,﻿ DESCRIPTION (provided by applicant): Oncol...,2016,1,1,3293,"[oncolytic_herpes_simplex_virus, hsv, promise,...",Adult; adult stem cell; Apoptosis; Apoptotic;...,IN VIVO IMAGING OF STEM CELL LOADED ONCOLYTIC ...,...,BOSTON,MA,021142696,UNITED STATES,3/1/2016,1/1/2017,93.395,2016,374428.0,NaN
98,857180,833135,﻿ DESCRIPTION (provided by applicant): The c...,2016,1,1,3129,"[career, k01, candidate, independent, preventi...",abeta accumulation; Affect; aging brain; Alle...,INFLUENCE OF GENETIC RISK FACTORS ON BIOMARKER...,...,BOSTON,MA,021142696,UNITED STATES,5/15/2016,1/1/2017,93.866,2016,131760.0,NaN


# Functions

In [6]:
def get_corpus_embeddings(dir):
    with open(dir) as f:
        ai_text = f.read()
    ai_corpus = tokenize.sent_tokenize(ai_text) #sentence tokenization
    ai_embeddings = embedder.encode(ai_corpus, show_progress_bar=True) # embeddings
    return ai_embeddings


# k: number of similar sentences from AI corpus
# abstract: abstract from FEDERAL RePORTER
# print_result: if TRUE, print out the similar sentenses from AI corpus to each sentence in the abstract
def get_score(k, abstract, print_result = False):
    queries = tokenize.sent_tokenize(abstract) 

    # init a result list for scores
    result = []
    
    # Find the closest k sentences of the AI corpus for each query sentence (ML) based on cosine similarity
    top_k = min(k, len(ai_embeddings))
    
    for query in queries: #compare each sentence in the abstract to the ai corpus
        query_embedding = embedder.encode(query, show_progress_bar=False) 
        
        # We use cosine-similarity and torch.topk to find the highest k scores
        cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
        
        top_results = torch.topk(cos_scores, k=top_k)   #get the top k scores
        result.append(top_results.values.tolist()) #unlist the top result list
        if print_result:
            print("\n\n======================\n\n")
            print("Query:", query)
            print("Results:", top_results)
            print("\nTop k=5 most similar sentences in corpus:")
            for score, idx in zip(top_results[0], top_results[1]):
                print(ai_corpus[idx], "(Score: {:.4f})".format(score))
    return result

In [7]:
start_time = datetime.datetime.now()
print("Start time:", start_time)

ai_embeddings = get_corpus_embeddings("/project/class/bii_sdad_dspg/uva_2021/dspg21RnD/ai_wiki_text.txt")

abstracts_try = abstracts_try.assign(score= abstracts_try["ABSTRACT"].apply(lambda x: get_score(5,x,False)))

sentence_score= []
for abstract in abstracts_try["score"]:
    sentence_score.append([statistics.mean(i) for i in abstract])

abstracts_try["sentence_score"]=sentence_score
abstracts_try

end_time = datetime.datetime.now()
print("Finished calculating ", len(abstracts_try), "of", "embedding score at", end_time)
print("It took", end_time-start_time, "to run.")

Start time: 2021-07-09 22:33:31.603077


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Finished calculating  100 of embedding score at 2021-07-09 22:33:57.289060
It took 0:00:25.685983 to run.


In [12]:
abstracts_try = abstracts_try.assign(abstract_score= abstracts_try["sentence_score"].apply(lambda x: statistics.mean(x)))

In [14]:
abstracts_out = abstracts_try[["original index", "PROJECT_ID", "score", "sentence_score", "abstract_score"]]

In [15]:
abstracts_out

,original index,PROJECT_ID,score,sentence_score,abstract_score
0,17608,152242,"[[0.30528321862220764, 0.2792899012565613, 0.2...","[0.24746134877204895, 0.2184512972831726, 0.19...",0.263356
1,111864,190316,"[[0.22555223107337952, 0.2161628156900406, 0.2...","[0.19784948378801345, 0.18860326260328292, 0.2...",0.246339
2,22052,154213,"[[0.4233601689338684, 0.4173330068588257, 0.41...","[0.38222160637378694, 0.3518401920795441, 0.29...",0.338063
3,35004,159362,"[[0.2699534595012665, 0.26451870799064636, 0.2...","[0.2295188069343567, 0.22275962084531784, 0.20...",0.262953
4,371628,594482,"[[0.39741647243499756, 0.3675093650817871, 0.3...","[0.31080458760261537, 0.2393555074930191, 0.23...",0.263024
...,...,...,...,...,...
95,758240,741964,"[[0.27936139702796936, 0.2665713131427765, 0.2...","[0.2600032642483711, 0.21491229981184007, 0.22...",0.245884
96,839161,915408,"[[0.27599045634269714, 0.2710220217704773, 0.2...",[0.24469317942857743],0.244693
97,884503,861883,"[[0.255683958530426, 0.2524286210536957, 0.249...","[0.24305053800344467, 0.24207402020692825, 0.2...",0.254868
98,857180,833135,"[[0.37934285402297974, 0.3674343526363373, 0.3...","[0.35079261660575867, 0.35246659219264986, 0.3...",0.346761


In [16]:
abstracts_out.to_csv(r'/project/class/bii_sdad_dspg/uva_2021/dspg21RnD/abstracts_embedding_score_try.csv', index = False)   


In [15]:
print("Finish at" , datetime.datetime.now())

Finish at 2021-07-09 19:30:19.394618
